In [1]:
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
print(os.getcwd())

from torchrbpnet.data import tfrecord_to_dataloader, dummy_dataloader

dataloader = tfrecord_to_dataloader('../example/head.tfrecord')

/home/marc/PhD/TorchRBPNet/notebooks
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
# %%
import torch
import torch.nn as nn
import pytorch_lightning as pl

import torchmetrics
from torchrbpnet.losses import MultinomialNLLLossFromLogits
from torchrbpnet.metrics import BatchedPCC, MultinomialNLLFromLogits
from torchrbpnet.networks import MultiRBPNet

# %%
class BatchIdx(torchmetrics.MeanMetric):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def update(self, batch_idx: torch.Tensor, *args, **kwargs):
        # update (i.e. take mean)
        super().update(batch_idx)

# %%
# class Model(pl.LightningModule):
#     def __init__(self, network):
#         super().__init__()
#         self.network = network
#         self.loss_fn = MultinomialNLLLossFromLogits()
#         self.metrics = nn.ModuleDict({'loss': MultinomialNLLFromLogits(), 'pcc': BatchedPCC()}) 
#         self.example_input_array = torch.rand(2, 4, 101)
#         self.test_metric = torchmetrics.MeanMetric() #BatchedPCC()

#         print(self.hparams)
    
#     def forward(self, *args, **kwargs):
#         return self.network(*args, **kwargs)

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         return optimizer

#     def training_step(self, batch, batch_idx, **kwargs):
#         x, y = batch
#         y_pred = self.forward(x)
#         loss = self.loss_fn(y, y_pred, dim=-2)
#         self.compute_and_log_metics(y_pred, y, partition='train')
#         return loss
    
#     def training_epoch_end(self, *args, **kwargs):
#         self._reset_metrics()

#     def validation_epoch_end(self, *args, **kwargs):
#         self._reset_metrics()

#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_pred = self.forward(x)
#         self.compute_and_log_metics(y_pred, y, partition='val')
    
#     def compute_and_log_metics(self, y_pred, y, partition=None):
#         on_step = False
#         if partition == 'train':
#             on_step = True

#         for name, metric in self.metrics.items():
#             metric(y_pred, y)
#             self.log(f'{partition}/{name}', metric.compute(), on_step=on_step, on_epoch=True, prog_bar=False)
    
#     def _reset_metrics(self):
#         for metric in self.metrics.values():
#             metric.reset()

class Model(pl.LightningModule):
    def __init__(self, network, _example_input_shape):
        super().__init__()
        self.network = network
        self.loss_fn = MultinomialNLLLossFromLogits()
        # self.metrics = nn.ModuleDict({'loss': MultinomialNLLFromLogits(), 'pcc': BatchedPCC()}) # This has to be wrapped in a nn.ModuleDict (otherwise .to_device has to be called manually on metrics)
        self.metrics = nn.ModuleDict({'loss': MultinomialNLLFromLogits(), 'pcc': BatchedPCC()})
        # self.example_input_array = torch.rand(*_example_input_shape)
    
    def forward(self, *args, **kwargs):
        return self.network(*args, **kwargs)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx, **kwargs):
        x, y = batch
        y_pred = self.forward(x)
        loss = self.loss_fn(y, y_pred, dim=-2)
        self.compute_and_log_metics(y_pred, y, partition='train')
        return loss
    
    def training_epoch_end(self, *args, **kwargs):
        self._reset_metrics()

    def validation_epoch_end(self, *args, **kwargs):
        self._reset_metrics()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        self.compute_and_log_metics(y_pred, y, partition='val')
    
    def compute_and_log_metics(self, y_pred, y, partition=None):
        on_step = False
        if partition == 'train':
            on_step = True

        for name, metric in self.metrics.items():
            metric(y_pred, y)
            self.log(f'{partition}/{name}', metric.compute(), on_step=on_step, on_epoch=True, prog_bar=False)
    
    def _reset_metrics(self):
        for metric in self.metrics.values():
            metric.reset()

model = Model(network=MultiRBPNet(n_tasks=223), _example_input_shape=next(iter(dataloader))[0].shape)

In [4]:
print(next(iter(dataloader))[1].shape)
print(model(next(iter(dataloader))[0]).shape)

torch.Size([5, 1000, 223])


/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,


torch.Size([5, 1000, 223])


In [5]:
import datetime
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, RichProgressBar

root_log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
loggers = [
    pl_loggers.TensorBoardLogger(root_log_dir+'/tensorboard', name='', version='', log_graph=True),
    # pl_loggers.CSVLogger(root_log_dir+'/tensorboard', name='', version=''),
]

checkpoint_callback = ModelCheckpoint(dirpath=f'{root_log_dir}/checkpoints', every_n_epochs=1, save_last=True)

early_stop_callback = EarlyStopping(monitor="val/loss", min_delta=0.00, patience=3, verbose=False, mode="min")

bar = RichProgressBar()

trainer = pl.Trainer(default_root_dir=root_log_dir, max_epochs=2, logger=loggers, callbacks=[checkpoint_callback, early_stop_callback])
trainer.fit(model=model, train_dataloaders=dataloader, val_dataloaders=dataloader)
# torch.save(model, 'model.pt')

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name    | Type                         | Params
---------------------------------------------------------
0 | network | MultiRBPNet                  | 1.9 M 
1 | loss_fn | MultinomialNLLLossFromLogits | 0     
2 | metrics | ModuleDict                   | 0     
---------------------------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.491     Total estimated model params size (MB)
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/loggers/tensorboard.py:188: UserWarning: Could not log computati

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1: : 2it [00:01,  1.26it/s, loss=49.1, v_num=]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: : 2it [00:01,  1.12it/s, loss=49.1, v_num=]


In [9]:
model

Model(
  (network): MultiRBPNet(
    (body): Sequential(
      (0): Conv1DFirstLayer(
        (conv1d): Conv1d(4, 256, kernel_size=(6,), stride=(1,), padding=same)
        (act): ReLU()
      )
      (1): Conv1DResBlock(
        (conv1d): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=same)
        (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
        (dropout): Dropout(p=0.25, inplace=False)
      )
      (2): Conv1DResBlock(
        (conv1d): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=same, dilation=(2,))
        (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): ReLU()
        (dropout): Dropout(p=0.25, inplace=False)
      )
      (3): Conv1DResBlock(
        (conv1d): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=same, dilation=(4,))
        (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [10]:
torch.save(model.network, 'test.pt')

InvalidArgumentError: Cannot convert a Tensor of dtype variant to a NumPy array.